# Gathering data from procyclingstats.com

This notebook scrapes the input data from [procyclingstats.com](https://www.procyclingstats.com/) using the [**procyclingsstats**](https://github.com/themm1/procyclingstats) scraping library. I add some high-level cleaning and assembling functionality on top to make the scraping easier.

It collects:
- For a large number of riders from the best teams...
- Metadata for each rider, but most importantly...
- Their results in one-day or multi-stage...
- High-level races...
- For up to a few years in the past

The data is transformed into a simple matrix (pandas DataFrame) format, so that it can be used in the next step's algorithm to find hidden factors (called embeddings) determining a racer's and a race's profile. All while having to specify rather little about the type of race! Ready, set, go!

_This is just for running along and testing, and does not generate the final results!_ A script version of this notebook is in `scripts/scrape.py`.

## Imports

In [1]:
import sys
import numpy as np
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from procyclingstats import (
    Race,          # Race("race/tour-de-france/2022/overview").parse()
    Rider,         # Rider("rider/tadej-pogacar").parse()
    Stage,         # Stage("race/tour-de-france/2018/stage-18").parse()
    Team,          # Team("team/bora-hansgrohe-2021").parse()
    RiderResults,  # RiderResults("rider/alberto-contador/results").parse()
    RaceStartlist,
    RaceClimbs,
    Ranking        # Ranking("rankings/me/individual").parse() --> Summation of PCS points over a 12-month + 2 weeks overlap period
)

sys.path.append("../")
from src.utils import *

## Config

In [2]:
YEARS = [2022, 2023]

In [3]:
CUTOFFDATE = "2023-09-18"
print(CUTOFFDATE)

2023-09-18


I use the 2023 races as base calendar, inluding only UCI Worldtour, UCI ProSeries, and Europe Tour races. Of course, races (and race names) change over the years but not so much. U23 (xU) and championships (NN/CC) races are dropped. I also had to remove a few duplicates. The idea is that we deduce the most important riders based on who participated in these races. Doing the inverse seems less straightforward with the API package.

In [4]:
df_races = pd.read_csv("../data/df_races.csv", delimiter=";", encoding="latin-1")
df_races = df_races.dropna()

In [5]:
df_races.Class.unique().tolist()  # 1.x = one-day race, 2.x = multi-day race & .UWT > .Pro > .1 > .2

['2.UWT', '1.UWT', '2.Pro', '1.Pro', '1.1', '1.2', '2.1', '2.2']

## Parse results

In [6]:
df_races_out_list = []
for year in YEARS:
    races, classes, stages = [], [], []
    for i, row in df_races.iterrows():
        race_key, _, race_class, race_slug = row
        
        race_slug_full = f"race/{race_slug}/{year}/overview"
        race_p = try_to_parse(Race, race_slug_full)     
        if race_p is None:
            continue
        else:
            # do not process if race end date is beyond dataset cutoff date
            # but keep going, because races are not ordered chronologically
            if race_p["enddate"] > CUTOFFDATE:
                print(f"Skipped: {race_slug_full}")
                continue
            
            stage_slug_base = race_slug_full.replace("/overview", "")  # has general classification if multi-stage race
            if race_p["is_one_day_race"] is True:
                stage_slugs = [f"{stage_slug_base}/result"]  # one-day race
            elif "stages" in race_p:
                stage_slugs = [f"{stage_slug_base}/gc"] + [f"{s['stage_url']}/result" for s in race_p["stages"]]  # multiple stages
            
            races += [race_key] * len(stage_slugs)
            classes += [race_class] * len(stage_slugs)
            stages += stage_slugs
    
    df_races_out_list.append(pd.DataFrame({"year": year, "race": races, "class": classes, "stage_slug": stages}))
        
df_races_out = pd.concat(df_races_out_list)

Oopsie! This one failed: race/vuelta-ciclista-a-la-provincia-de-san-juan/2022/overview
Oopsie! This one failed: race/grand-prix-aspendos/2022/overview
Oopsie! This one failed: race/grand-prix-apollon-temple-me/2022/overview
Oopsie! This one failed: race/figueira-champions-classic/2022/overview
Oopsie! This one failed: race/alanya-cup/2022/overview
Oopsie! This one failed: race/le-tour-des-100-communes/2022/overview
Oopsie! This one failed: race/syedra-ancient-city/2022/overview
Oopsie! This one failed: race/arno-wallaard-memorial/2022/overview
Oopsie! This one failed: race/giro-di-reggio-calabria/2022/overview
Oopsie! This one failed: race/gp-vorarlberg/2022/overview
Oopsie! This one failed: race/euregio-bodensee-rundfahrt/2022/overview
Oopsie! This one failed: race/gp-internacional-beiras-e-serra-da-estrela/2022/overview
Oopsie! This one failed: race/due-giorni-marchigiana-g.p.-santa-rita/2022/overview
Oopsie! This one failed: race/due-giorni-marchigiana-trofeo-citta-di-castelfidar/20

In [7]:
print(len(df_races_out))
df_races_out.head(15)

1548


,year,race,class,stage_slug
0,2022,Santos Tour Down Under,2.UWT,race/tour-down-under/2022/gc
1,2022,Santos Tour Down Under,2.UWT,race/tour-down-under/2022/stage-1/result
2,2022,Santos Tour Down Under,2.UWT,race/tour-down-under/2022/stage-2/result
3,2022,Santos Tour Down Under,2.UWT,race/tour-down-under/2022/stage-3/result
4,2022,Santos Tour Down Under,2.UWT,race/tour-down-under/2022/stage-4/result
5,2022,Santos Tour Down Under,2.UWT,race/tour-down-under/2022/stage-5/result
6,2022,Santos Tour Down Under,2.UWT,race/tour-down-under/2022/stage-6/result
7,2022,Cadel Evans Great Ocean Road Race,1.UWT,race/great-ocean-race/2022/result
8,2022,UAE Tour,2.UWT,race/uae-tour/2022/gc
9,2022,UAE Tour,2.UWT,race/uae-tour/2022/stage-1/result


In [8]:
df_races_out["parsed"] = df_races_out["stage_slug"].apply(lambda x: try_to_parse(Stage, x))

Oopsie! This one failed: race/vuelta-a-espana/2022/gc
Oopsie! This one failed: race/vuelta-a-espana/2022/stage-21/result
Oopsie! This one failed: race/trofej-umag-umag-trophy/2022/result
Oopsie! This one failed: race/porec-trophy-trofej-porec/2022/result
Oopsie! This one failed: race/istarsko-proljece-istrian-spring-trophy/2022/prologue/result
Oopsie! This one failed: race/istarsko-proljece-istrian-spring-trophy/2022/stage-1/result
Oopsie! This one failed: race/istarsko-proljece-istrian-spring-trophy/2022/stage-2/result
Oopsie! This one failed: race/east-midlands-international-cicle-classic/2022/result
Oopsie! This one failed: race/trofeo-citta-di-brescia/2022/result
Oopsie! This one failed: race/visegrad-4-bicycle-race-gp-slovakia/2022/result
Oopsie! This one failed: race/memorial-andrzeja-trochanowskiego/2022/result
Oopsie! This one failed: race/tour-of-bulgaria/2022/prologue/result
Oopsie! This one failed: race/tour-of-bulgaria/2022/stage-1/result
Oopsie! This one failed: race/gylne

In [9]:
# handy to keep track of issues
stages_not_parsed = df_races_out[df_races_out.parsed.isnull()]["stage_slug"].tolist()
print(f"{len(stages_not_parsed)} out of {len(df_races_out)} race results were not parsed")

72 out of 1548 race results were not parsed


In [10]:
df_races_out.dropna(subset=["parsed"], inplace=True)  # drop stages that couldn't be parsed

In [11]:
df_races_out["results"] = df_races_out[["stage_slug", "parsed"]].apply(lambda x: parse_results_from_stage(*x), axis=1)

In [12]:
df_races_out.shape

(1476, 6)

In [13]:
vec = DictVectorizer()

measurements = df_races_out["results"].apply(lambda x: {} if x is None else dict(x))
df_results = pd.DataFrame(
    vec.fit_transform(measurements).toarray(),
    columns=vec.get_feature_names_out(),
    # set year, stage slug, and class as indices
    index=pd.MultiIndex.from_frame(pd.concat([df_races_out["year"],
                                              df_races_out["stage_slug"].str.replace("race/", ""),
                                              df_races_out["class"]],
                                             axis=1))
)

df_results.replace(0, np.nan, inplace=True)  # initially NaN = did not finish race, 0 = did not participate; this replace() drops distinction

In [14]:
df_results.sample(5)

AAGAARD HANSEN Tobias  \
year stage_slug                                                       class                          
2023 uae-tour/2023/gc                                                 2.UWT                    NaN   
2022 tour-cycliste-international-de-la-guadeloupe/2022/stage-2/result 2.2                      NaN   
     tour-de-pologne/2022/stage-6/result                              2.UWT                    NaN   
     tour-du-doubs/2022/result                                        1.1                      NaN   
2023 dookola-mazowsza/2023/gc                                         2.2                      NaN   

                                                                             AALRUST Håkon  \
year stage_slug                                                       class                  
2023 uae-tour/2023/gc                                                 2.UWT            NaN   
2022 tour-cycliste-international-de-la-guadeloupe/2022/stage-2/result 2.2              NaN   
     tour-de-pologne/2022/stage-6/result                              2.UWT            NaN   
     tour-du-doubs/2022/result                                        1.1              NaN   
2023 dookola-mazowsza/2023/gc                                         2.2            121.0   

                                                                             AASHEIM Anders Svardal  \
year stage_slug                                                       class                           
2023 uae-tour/2023/gc                                                 2.UWT                     NaN   
2022 tour-cycliste-international-de-la-guadeloupe/2022/stage-2/result 2.2                       NaN   
     tour-de-pologne/2022/stage-6/result                              2.UWT                     NaN   
     tour-du-doubs/2022/result                                        1.1                       NaN   
2023 dookola-mazowsza/2023/gc                                         2.2                       NaN   

                                                                             AASKOV PALLESEN Jeppe  \
year stage_slug                                                       class                          
2023 uae-tour/2023/gc                                                 2.UWT                    NaN   
2022 tour-cycliste-international-de-la-guadeloupe/2022/stage-2/result 2.2                      NaN   
     tour-de-pologne/2022/stage-6/result                              2.UWT                    NaN   
     tour-du-doubs/2022/result                                        1.1                      NaN   
2023 dookola-mazowsza/2023/gc                                         2.2                      NaN   

                                                                             AASVOLD Kristian  \
year stage_slug                                                       class                     
2023 uae-tour/2023/gc                                                 2.UWT               NaN   
2022 tour-cycliste-international-de-la-guadeloupe/2022/stage-2/result 2.2                 NaN   
     tour-de-pologne/2022/stage-6/result                              2.UWT               NaN   
     tour-du-doubs/2022/result                                        1.1                 NaN   
2023 dookola-mazowsza/2023/gc                                         2.2                 NaN   

                                                                             ABAY Burak  \
year stage_slug                                                       class               
2023 uae-tour/2023/gc                                                 2.UWT         NaN   
2022 tour-cycliste-international-de-la-guadeloupe/2022/stage-2/result 2.2           NaN   
     tour-de-pologne/2022/stage-6/result                              2.UWT         NaN   
     tour-du-doubs/2022/result                                        1.1           NaN   
2023 dookola-mazowsza/2023/gc           

In [15]:
df_results.filter(regex="VAN AERT Wout").dropna().loc[2022]

,,VAN AERT Wout
stage_slug,class,
omloop-het-nieuwsblad/2022/result,1.UWT,1.0
paris-nice/2022/gc,2.UWT,32.0
paris-nice/2022/stage-1/result,2.UWT,3.0
paris-nice/2022/stage-2/result,2.UWT,2.0
paris-nice/2022/stage-3/result,2.UWT,3.0
paris-nice/2022/stage-4/result,2.UWT,1.0
paris-nice/2022/stage-5/result,2.UWT,98.0
paris-nice/2022/stage-6/result,2.UWT,3.0
paris-nice/2022/stage-7/result,2.UWT,62.0


In [16]:
print(df_results.shape)
df_results = df_results.dropna(axis=0, how="all")  # drop results that couldn't be parsed
print(df_results.shape)

(1476, 5887)
(1397, 5887)


In [17]:
df_results.columns = [clean_rider_name(c) for c in df_results.columns.str.strip()]

## Parse riders data

In [18]:
# df_results = pd.read_csv("../data/df_race_results.csv")

In [19]:
riders_all = sorted(df_results.columns)

In [20]:
# birth_dates, nationalities = [], []
# for rider_name in riders_all:
#     rider_slug = convert_name_to_slug(rider_name)
#     try:
#         rider = Rider(f"rider/{rider_slug}")
#         birth_dates.append(rider.birthdate())
#         nationalities.append(rider.nationality())
#     except (ValueError, AttributeError):
#         print(f"Damn! Rider not found: {rider_name} --> {rider_slug}")
#         birth_dates.append(None)
#         nationalities.append(None)
#         continue

In [21]:
# df_riders = pd.DataFrame({
#     "name": riders_all,
#     "birth_date": birth_dates,
#     "nationality": nationalities
# })

In [22]:
df_riders = pd.DataFrame(
    {
        "rider_name": riders_all,
        "rider_slug": [convert_name_to_slug(r) for r in riders_all],
    }
)
df_riders[["nationality", "birth_date"]] = (
    df_riders["rider_slug"].apply(parse_rider_info).tolist()
)
df_riders.drop(columns=["rider_slug"], inplace=True)
print(f"{df_riders.nationality.isnull().sum()} out of {len(df_riders)} riders' metadata was not parsed")
df_riders.dropna(inplace=True)

306 out of 5887 riders' metadata was not parsed


In [23]:
df_riders

,rider_name,nationality,birth_date
0,AAGAARD HANSEN Tobias,DK,2002-3-10
1,AALRUST Håkon,NO,1998-1-5
2,AASHEIM Anders Svardal,NO,2002-5-7
4,AASVOLD Kristian,NO,1995-5-30
5,ABAY Burak,TR,1996-1-1
...,...,...,...
5882,ŤOUPALÍK Jakub,CZ,2001-7-17
5883,ŻELAZOWSKI Michał,PL,2004-8-6
5884,ŻUREK Jakub,PL,2002-1-28
5885,ŽUMER Matic,SI,1997-11-19


## Merge and store data

In [24]:
df_results = df_results[[r for r in df_results.columns if r in df_riders.rider_name.tolist()]]

In [25]:
df_riders.shape[0], df_results.shape[1]

(5581, 5581)

In [26]:
df_riders.to_csv("../data/df_riders_data.csv", index=False)

In [27]:
df_results.to_csv("../data/df_race_results.csv", index=True)